In [ ]:
!pip -q install diffusers transformers scipy segment_anything

In [ ]:
from diffusers import StableDiffusionInpaintPipeline
from PIL import Image
import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = "cuda"

In [ ]:
pipe = StableDiffusionInpaintPipeline.from_pretrained("stabilityai/stable-diffusion-2-inpainting", torch_dtype=torch.float16,).to(device)

In [ ]:
import os
original_dir = "/kaggle/input/dataset1/images/img_111_jpg.rf.26dc0a9a0a902220b60b5f40ecedf994.jpg"
mask_dir = "/kaggle/input/dataset1/TEST/img_111_jpg.rf.26dc0a9a0a902220b60b5f40ecedf994.png"
output_dir = "/kaggle/working/result1"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd GroundingDINO
!pip -q install -e .
from groundingdino.util.inference import load_model, load_image, predict, annotate

In [ ]:
src, img = load_image(original_dir)
original_img = Image.fromarray(src).resize((512, 512))

In [ ]:
import os
from PIL import Image
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
input_dir = "/kaggle/input/dataset2/images/"  
mask_dir = "/kaggle/input/dataset2/TRAIN/"   
output_dir = "/kaggle/working/output2/"
model_id = "stabilityai/stable-diffusion-2"
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")

pipe = StableDiffusionInpaintPipeline.from_pretrained(
    model_id,scheduler=scheduler, torch_dtype=torch.float16
)
pipe = pipe.to("cuda")  

prompt = "yellow smoke"
negative_prompt = "black or grey smoke"

os.makedirs(output_dir, exist_ok=True)
input_files = os.listdir(input_dir)

for filename in input_files:
  img_path = os.path.join(input_dir, filename)
  mask_path = os.path.join(mask_dir, filename.replace(".jpg", ".png"))
  if not os.path.isfile(mask_path):
    print(f"Skipping {filename}: Mask image not found at {mask_path}")
    continue 
  original_img = Image.open(img_path)
  mask_img = Image.open(mask_path)
  original_img = original_img.resize((512, 512))
  mask_img = mask_img.resize((512, 512))
  edited_image = pipe(prompt=prompt,negative_prompt=negative_prompt,guidance_scale=15, image=original_img, mask_image=mask_img).images[0]
  output_path = os.path.join(output_dir, filename)
  edited_image.save(output_path)

print("Inpainting process complete. Edited images saved to:", output_dir)

In [ ]:
prompt = "yellow smoke"
image = pipe(prompt=prompt, image=original_img, mask_image=mask_img).images[0]
image.save("/kaggle/working/a.jpg")

In [ ]:
!zip -r output1.zip /kaggle/working/output